In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
df = pd.read_excel('xlsx_file_path', header=None)
# nan_values = df.isnull().any()
# print(nan_values)
## drroping the column
# df=df[df.columns[df.isna().any()]]
## dropping row
null_data = df[df.isnull().any(axis=1)]
# print(null_data)
data_top=null_data.index
index=[]
for i in data_top:
    index.append(i)
print(index)
df=cdist(df,df,'euclid')
df=pd.DataFrame(df)
# print(df.to_string())
#similarity dataset
#df


In [ ]:
#converting into a numpy array and fixing the threshold
a= np.array(df)
med = 0.785
print(med)

In [ ]:
#keeping only values above threshold and making others as 0.
similarity_thres = []
for i in a:
    f1=[]
    for k in i:
        if(k==1):
            f1.append(0)
        elif(k>=med):
            f1.append(1)
        else:
            f1.append(0)
    similarity_thres.append(f1)

In [ ]:
#printing the boolean matrix
# similarity_thres

In [ ]:
#creating a new dataframe using the boolean matrix
new_df = pd.DataFrame(similarity_thres)

In [ ]:
#creating an index column to get the label in the desired format.
new_df['DOC'] = (new_df.index)
new_df['X']= "X"
new_df['DOC2'] = new_df['X']+new_df['DOC'].astype(str)
del new_df['DOC'],new_df['X']

In [ ]:
# #plotting the network 
# import networkx as nx
# import matplotlib.pyplot as plt
# stacked = new_df.set_index(['DOC2']).stack()
# stacked = stacked[stacked==1]
# edges = stacked.index.tolist()
# #print(il1)
# #edges
# G = nx.Graph(edges)
# Gp = nx.bipartite.project(G,new_df.set_index('DOC2').columns)
# Gp.edges()
# #pos = {node:[0, i] for i,node in enumerate(il1['DOC2'])}
# #pos.update({node:[1, i] for i,node in enumerate(il1['level_1'])})
# nx.draw(G, with_labels=True)
# #for p in pos:  # raise text positions
#  #   pos[p][1] += .25
# #nx.draw_networkx_labels(G,pos=pos)

# plt.show()

In [ ]:
##Reading the original dataset and performing the same to create labels and reindex.
dataset = pd.read_excel('xlsx_file_path', header=None)
dataset.index = dataset.index+1
dataset['DOC'] = (dataset.index)
dataset['X']= "X"
dataset['index'] = dataset['X']+dataset['DOC'].astype(str)
del dataset['DOC'],dataset['X']
print(dataset)

In [ ]:
#splitting the null values and creating a new dataframe using the similarity boolean matrix.
#dataset.set_index('index',inplace=True)
dataset
null_data = dataset[dataset.isnull().any(axis=1)]
similarity_bool = new_df.copy()
similarity_bool.set_index('DOC2',inplace=True)

In [ ]:
#Resetting the index and deleting the index column
similarity_bool.reset_index(inplace=True)
#del similarity_bool['index']

In [ ]:
#del similarity_bool['level_0']

In [ ]:
## Taking only values above threshold but instead of converting it into 0 & 1 , we are keeping the similarity score as such.
similarity_thres1 = []
for i in a:
    f1=[]
    for k in i:
        if(k==1):
            f1.append(0)
        elif(k>=med):
            f1.append(k)
        else:
            f1.append(0)
    similarity_thres1.append(f1)

In [ ]:
##sscore is a new dataframe with the above scores.
sscore = pd.DataFrame(similarity_thres1)
sscore.index = sscore.index+1
sscore['DOC'] = (sscore.index)
sscore['X']= "X"
sscore['DOC2'] = sscore['X']+sscore['DOC'].astype(str)
del sscore['DOC'],sscore['X']
# sscore.set_index('DOC2',inplace=True)
# print(sscore)

In [ ]:
#doing stacked to only take the values not equal to 0.0 and then unstacking it.
stacked1 = sscore.set_index(['DOC2']).stack()
#print(stacked1)
stacked1 = stacked1[stacked1 != 0.0]
# print(stacked1)
edges = stacked1.index.tolist()
# print(edges)
unstacked = stacked1.unstack()
# print(unstacked)

In [ ]:
#creating the incomplete_tuples and complete_tuples from the unstacked dataframe.
print(unstacked)
##To determine the P value
incomplete_tuples = unstacked.columns.to_list()
total_tuples = unstacked.index.to_list()
complete_tuples = list(set(total_tuples) - set(incomplete_tuples))
print(incomplete_tuples)
print(complete_tuples)

In [ ]:
#creating a new column in unstacked to house the index values.
unstacked['indexed'] = unstacked.index
#filling the unstacked df for NaN
unstacked.fillna(0,inplace=True)
#del unstacked['index']

In [ ]:
#Keeping only complete neighbours for the incomplete tuples in this "inc_complete" dataframe.
inc_complete = unstacked[~unstacked['indexed'].isin(incomplete_tuples)]
print(inc_complete)
#inc_complete
#Keeping only incomplete neighbours for the incomplete tuples in this "inc_complete" dataframe.
inc_incomplete =unstacked[unstacked['indexed'].isin(incomplete_tuples)]
print(inc_incomplete)

In [ ]:
#Function to get complete neighbours and their similarities in a dictionary
#iterating over the columns in inc_complete, retrieving the complete neighbours for each incomplete tuple and putting them in a dictionary.
def incomplete_connection_creator(list_1,dict_1):
  listed = []
  listed1= []
  inc_complete = list_1
  incomplete_tuples = dict_1
  for column in inc_complete:
    name = column
    #print(name)
    kailai = (list(inc_complete[inc_complete[column] != 0.0].indexed))
    kailai1 = (list(inc_complete[inc_complete[column] != 0.0][column]))
    #print(kailai)
    listed.append(kailai)
    listed1.append(kailai1)
  #print(listed)
  zipped = zip(incomplete_tuples,listed)
  zipped1 = zip(incomplete_tuples,listed1)
  dict_da = dict(zipped)
  dict_db = dict(zipped1)
  return dict_da,dict_db

In [ ]:
#Getting the complete neighbours and their similarity scores
data_dict,data_dict1 = incomplete_connection_creator(inc_complete,incomplete_tuples)
# print(data_dict,"\n",data_dict1)

In [ ]:
##Calculating the p value for the final calculation of node weights.
import math
def p_value_calc(list_1,dict_1):
  incomplete_tuples = list_1
  data_dict = dict_1
  ##calculating the term (1/Xm)
  if len(incomplete_tuples) != 0:
    term = 1/(len(incomplete_tuples))
  else:
    term = 1
  ##Summing all the length of values present in the dictionary to get the total number of complete neighbours to the incomplete tuples.
  summing = sum(len(v) for v in data_dict.values())
  ##math.ceil to Round up to the next whole number for the product of summing and term.
  value_1 =math.ceil(term * summing)
  return value_1

In [ ]:
#P-value :
p_value = p_value_calc(incomplete_tuples,data_dict)
print(p_value)

In [ ]:
##Creating a dictionary to find out the number of similarity scores associated with each incomplete tuple. We get the length of the dictionary's values for a particular element.
def length_getter(list_a,dict_a):
  list_length = []
  incomplete_tuples = list_a
  data_dict1 = dict_a
  ## checking if element in incomplete_tuples
  for y in incomplete_tuples:
    ##for that element we check the similarity scores in data_dict1 dictionary
    for y in data_dict1:
      ##intializing total to 0
      total = 0
      #print("Element:",y)
      #referencing to fetch the similarity scores -> ex: data_dict1['X1']=[0.814,0.836]
      value_list = data_dict1[y]
      #print(value_list)
      ##Taking their length
      count = len(value_list)
      total += count
      ##appending individual length to a list
      list_length.append(total)
  #print(list_length)
  ##Creating the dictionary which contains the incomplete tuples and their associated value lengths.
  zipper=zip(incomplete_tuples,list_length)
  data_dict = dict(zipper)
  return data_dict

In [ ]:
data_dict2 = length_getter(incomplete_tuples,data_dict1)
# print(data_dict2)

In [ ]:
##Calculating the node weights and putting them in a dictionary function
def node_weight_calculator(dict_1,dict_2,tuple_1,value1):
  list_scores = [] 
  diction_1 = dict_1
  diction_2 = dict_2 
  tupled_1 = tuple_1 
  p_value = value1
  ## iterating over incomplete tuple  present in incomplete_tuples list.
  for ij in tupled_1:
    #calculating the length of each item in data_dict2
    length = (dict_2[ij])
    #if length is greater than 0 we calculate the similarity score, else we assign 0
    if length>0:
      multiply_list = []
      #iterate over the length so as to capture all the associate similarity pairs with each incomplete tuple.
      for i in range(length):
        sim_score = dict_1[ij][i]
        ##Calculating the part of node weight from the formula given in the paper.
        initial_calc = (1-sim_score)**(1/(p_value))
        ##appending to multiply_list the values of initial_calc
        multiply_list.append(initial_calc)
      #Multiplying all the initial_Calc associated with a element.
      weight_1 =np.prod(multiply_list)
      ##subtracting 1 from weight_1 as per formula and rounding it to three decimal places to get the final node weight.
      final_weight = np.round(1 - weight_1,3)
    else:
      #keeping node weight as 0 if there is no similarity
      final_weight = 0.0
    list_scores.append(final_weight)
  ##Creating the final dictionary which contains the incomplete tuple and its associated node weight.
  zipper_score = zip(tupled_1,list_scores)
  dict_output = dict(zipper_score)
  return dict_output

In [ ]:
##Initial Node weight calculator
dict_scores = node_weight_calculator(data_dict1,data_dict2,incomplete_tuples,p_value)
print("Initial Node weight beofre Imputation Order calc:\n",dict_scores)

In [ ]:
##Incomplete- Incomplete connections and similarities
data_dict_inc,data_dict_inc_1 = incomplete_connection_creator(inc_incomplete,incomplete_tuples)
print("Incomplete neighbours to incomplete tuples:\n",data_dict_inc)
print("Incomplete neighbours similarity Score:\n",data_dict_inc_1)

In [ ]:
## Calculating the Node Weight like we did previously for the values.
##Creating a dictionary to find out the number of similarity scores associated with each incomplete tuple. We get the length of the dictionary's values for a particular element.
data_dict_2 = length_getter(incomplete_tuples,data_dict_inc_1)
print("Length of incomplete neighbours:\n",data_dict_2)

##Appending the node weight we have already calculated for these nodes.
print("Node Weights:",dict_scores)
score_list = []
score_list_name = []
removed_item =[]
for element in incomplete_tuples:
  if data_dict_2[element] > 0:
    score_list_name.append(element)
    score_list.append(dict_scores[element])    
  else:
    print("Removed")
    removed_item.append(element)
zip_score_final = zip(score_list_name,score_list)
data_final_scores = dict(zip_score_final)
print("removed item",removed_item)
print("Node Weights input to Greedy Algorithm:\n",data_final_scores)

In [ ]:
def incomplete_connection_creator1(list_1,dict_1):
  listed = []
  listed1= []
  inc_complete = list_1
  incomplete_tuples = dict_1
  #print(inc_complete)
  #print(incomplete_tuples)
  for column in inc_complete:
    for column in incomplete_tuples:
      name = column
      #print(name)
      kailai = (list(inc_complete[inc_complete[column] != 0.0].indexed))
      kailai1 = (list(inc_complete[inc_complete[column] != 0.0][column]))
      #print(kailai)
      listed.append(kailai)
      listed1.append(kailai1)
    #print("listed:",listed)
    zipped = zip(incomplete_tuples,listed)
    zipped1 = zip(incomplete_tuples,listed1)
    dict_da = dict(zipped)
    dict_db = dict(zipped1)
    return dict_da,dict_db

In [ ]:
'''
GREEDY ALGORITHM TO FIND THE IMPUTATION ORDER
STEP 1: Find Node with Largest Weight. Keep it as the starting point
STEP 2: Calculate the Filling Gain to make it a complete Neighbour.
STEP 3: If the imputed Node, contains any connections to incomplete tuples then update their node weights by considering the imputed node
        as a completed neighbour.
STEP 4: Repeat this process untill all the incomplete tuples are imputed.
'''

##STEP 1: Find Node with Largest Weight. Keep it as the starting point
#Getting the data_final_scores
incomplete_tuples_score = data_final_scores
print("Incomplete Tuple Scores Before Iteration:",incomplete_tuples_score)
#updated_incomplete_tuples = incomplete_tuples.copy()
updated_incomplete_tuples = score_list_name.copy()
updated_complete_tuples = complete_tuples.copy()
final_greedy_order = []
print("Node Weight for incomplete tuples stored in a new dictionary:",incomplete_tuples_score)
################################################################################################
##########################:
for i in range(len(incomplete_tuples_score)):  
  max_value = max(incomplete_tuples_score, key=incomplete_tuples_score.get)
  print("Node With the largest weight:",max_value)
  print("Node weight:",incomplete_tuples_score[max_value])
  
  ##Calculate the Filling Gain
  #print("X is:",data_dict_2[max_value])
  #print("Pairs are :",data_dict_inc[max_value])

  # data_dict_inc.pop('X6',None)
  # data_dict_2.pop('X6',None)
  # print(data_dict_2)
  # print(incomplete_tuples_score)
  if max_value in data_dict_2:
    list_score_holder = []
    final_gain_score_holder = []  
    for i in range(data_dict_2[max_value]):
      try:
       w_label = data_dict_inc[max_value][i]
       w_score = incomplete_tuples_score[w_label]
       print("Connected pair and score :",w_label,w_score)
       list_score_holder.append(w_score)
       #print(data_dict_inc_1[max_value][i])
       #print(p_value)
       w_difference = np.round((incomplete_tuples_score[max_value] - list_score_holder[i]),3)
       #print(data_dict_inc_1[max_value][i])
       w_second_part =np.round((1-data_dict_inc_1[max_value][i])**(1/p_value),3)
       gain = np.round(((w_difference)*(1-w_second_part)),3)
       #print(w_difference)
       #print(w_second_part)
       #print("Gain is :",gain)
       final_gain_score_holder.append(gain)
       #print("Final Gain calculated:",final_gain_score_holder)
      except Exception as e:
       print(e)

    gain_for_element = sum(final_gain_score_holder)
    print("Gain For incomplete tuple:",gain_for_element )
    updated_incomplete_tuples.remove(max_value)
    updated_complete_tuples.append(max_value)
    print("Updated_incomplete_tuples",updated_incomplete_tuples)
    print("Updated complete tuples",updated_complete_tuples)
    inc_complete_updated = unstacked[~unstacked['indexed'].isin(updated_incomplete_tuples)]
    data_dict_upd_1,data_dict_upd_2 = incomplete_connection_creator1(inc_complete_updated,updated_incomplete_tuples)
    upd_p_value = p_value_calc(updated_incomplete_tuples,data_dict_upd_1)
    data_dict_upd = length_getter(updated_incomplete_tuples,data_dict_upd_2)
    print("data_dict_upd_2:-",data_dict_upd_2)
    print("data_dict_upd_1:",data_dict_upd_1)
    print(max_value)
    new_dict = { k : v for k,v in data_dict_upd_1.items() if max_value in v}
    new_dict_scores = []
    new_dict_key = []
    for key,value in new_dict.items():
      new_value = data_dict_upd_2[key]
      new_dict_scores.append(new_value)
      new_key = key
      new_dict_key.append(key)
    new_zipped = zip(new_dict_key,new_dict_scores)
    new_dict_input = dict(new_zipped)
    print(new_dict_input)
    dict_scores_upd = node_weight_calculator(new_dict_input,data_dict_upd,new_dict_key,p_value)
    print("Updated Node weight before Imputation Order calc:\n",dict_scores_upd)
    dict_scores_upd.update({max_value:0.0})
    final_greedy_order.append(max_value)
    print("Appended:",dict_scores_upd)
    incomplete_tuples_score.update(dict_scores_upd)
    print("Updated Incomplete Tuples scores before proceeding to next iter",incomplete_tuples_score)
    print("----------------------------------------------------------------------------------------------------------------------")
print("Final Greedy Imputation order is",final_greedy_order)

##Creating the final order for feeding to the KNN
final_order = removed_item+final_greedy_order
print("final_order",final_order)
final_greedy_impute = []
# for a in final_order:
#   str_name = str(a)
#   strip_name = str_name[1:]
#   final_greedy_impute.append(int(strip_name))
data_top= np.array(data_top)
data_top=data_top+1
final_greedy_impute= data_top
# print(data_top)
print(final_greedy_impute)

In [ ]:
############################
#KNN Modelling Approach
dataset_1 = dataset.copy()
# dataset_1 
dataset_1.set_index('index',inplace=True)
dataset_1

In [ ]:
##Order Imputed Sequential KNN 
##Creating a null empty array equal to the dataset size
dataset_array = dataset_1.to_numpy()
print("Dataset Converted into a Numpy Array:\n",dataset_array)
#imputed_data = np.zeros([len(dataset_1),len(dataset_1.columns)])
imputed_data = np.zeros_like(dataset_array)
print("\nZeroes Array\n",imputed_data)
#Creating Empty lists 
complete = []
incompleted = []
missing =[]
com_ind=[]
incomp_ind=[]
##Taking the rows and cols of the input dataset for iteration purpose
[rows,cols] = dataset_array.shape
print("\nNo of Rows are",rows)
print("No of Cols are",cols)
##iterating through the number of rows
for i in range(len(dataset_array)):
  a = dataset_array[i]
  #print("Row is ",a)
  nan_checker = np.sum(np.sum(a, axis=0))
  #print("Sum of Row is",nan_checker)
  if math.isnan(nan_checker) :
    #print("Spotted Nan Value - ",nan_checker)
    incompleted.append(a)
    incomp_ind.append(i+1)
    #print("Non Sum of non numbers",np.nansum(np.nansum(a, axis=0)))
  else:
    complete.append(a)
    com_ind.append(i+1)
zipped_com = zip(com_ind,complete)
dict_complete = dict(zipped_com)
##Ordering the list as per imputation order generated from Greedy
zipped_incom = zip(incomp_ind,incompleted)
dict_incompleted = dict(zipped_incom)
print(dict_incompleted)
sorted_incomplete =[(v,dict_incompleted[v]) for v in final_greedy_impute]
print(sorted_incomplete)
incom_ind = []
incomplete = []
for header in range(len(sorted_incomplete)):
  header_app = sorted_incomplete[header][0]
  value_app = sorted_incomplete[header][1]
  incom_ind.append(header_app)
  incomplete.append(value_app)
print(incom_ind)
print(incomplete)
incom_ind_1 = sorted_incomplete[0][0]
print(incom_ind_1)
for i in com_ind:
  imputed_data[i-1] = dict_complete[i]
print("Imputed Data After Filling the Complete Rows:\n",imputed_data)
print("Incomplete Rows in the Array\n",incomplete)

for k in incomplete:
  #print(k)
  dist = []
  cgen = len(complete)
  for j in range(cgen):
    #print(k)
    #print("j",j)
    #print(complete[j])
    d = np.nansum(np.power(([k]-complete[j]),2))
    dist.append(d)
  print("Distance Before Sort:",dist)  
  position=[]
  for l in range(len(dist)):
    position.append(l)
  zipped_pos = zip(position,dist)
  dict_pos = dict(zipped_pos)
  #print(dict_pos)
  from operator import itemgetter  
  dict_pos_sorted =  sorted(dict_pos.items(), key = itemgetter(1), reverse = False)
  print("Sorted Dictionary",dict_pos_sorted)
  K = 5
  cluster_dictionary = dict_pos_sorted[:5]
  print("K Cluster Dictionary",cluster_dictionary)
  k_dist = []
  k_weight = []
  k_final_weight = []
  for d,s in cluster_dictionary:
    k_dist.append(s)
  print("Retrieved K Distances",k_dist)
  #total_list_sum = sum(1/((k_dist)))
  #print(total_list_sum)
  for h in range(len(k_dist)):
    #weight = np.true_divide((1/k_dist[h]),)
    individual_weight = ((1/k_dist[h]))
    k_weight.append(individual_weight)
  for u in k_weight:
    final_weight = np.true_divide(u,sum(k_weight))
    k_final_weight.append(final_weight)
  #print("K Weight:",k_weight)   
  print("Final_Weight:",k_final_weight)
  #print(complete)
  for a in range(np.size(dataset_array,1)):
      complete_values_list =[] 
      column_value = k[a]
      if math.isnan(column_value):
        for o,p in cluster_dictionary:
          complete_value = complete[o][a]
          complete_values_list.append(complete_value)
        #print("Complete_value",complete_values_list)
        imputed_final_value = round(sum(f * e for f, e in zip(complete_values_list,k_final_weight)),3)
        print("-----------------------------------------------------------------------------------------")
        print("Final Imputed value:",imputed_final_value) 
        print("-----------------------------------------------------------------------------------------")
        k[a] = imputed_final_value
  complete.append(k)
incomplete_dataframe = pd.DataFrame(incomplete)
incomplete_dataframe['ind'] = incom_ind
incomplete_dataframe.set_index('ind',inplace=True)
imputed_data_frame = pd.DataFrame(imputed_data)
imputed_data_frame.index = imputed_data_frame.index+1
final_imputed_df = incomplete_dataframe.combine_first(imputed_data_frame)
final_imputed_df.columns = dataset_1.columns
print("------------------------------------------------------------------------")
print("------------------------------------------------------------------------")
print("Final Imputed Dataset:\n", final_imputed_df)
print("------------------------------------------------------------------------")
print("------------------------------------------------------------------------")
print("Completed")
final_imputed_df.to_excel('Destination_folder_path_with_file_name_# example
/Users/Downloads/imputed_Ionosphere_AE_1.xlsx')
